#### Задача 1 (3 балла). 

Разовьем тему с бойцами. Напишите игру (можно взять свой старый код в качестве базы), где игроку будет предложено выбрать класс героя: волшебник или боец. Вы должны учесть возможность добавления новых игровых классов (используйте наследование). У волшебника и бойца немного разные атрибуты (можно атрибуты сделать одинаковые в классе-родителе, но разные коэффициенты в классах-детях, на которые они домножаются: например, здоровье волшебника будет 1.0 от стандартного значения, а здоровье бойца - 1.5, а с маной наоборот). Также у них будут разные методы "нанести удар" и, если хотите, приветствия. Также у нашего героя, кем бы он ни был, должен быть рюкзак, в котором можно рыться и хранить ограниченный набор вещей (в частности, там хранятся зелья: по умолчанию пусть в начале игры каждому персонажу дается по три зелья). Наконец, нужен класс для противника: можете придумать любого монстра (тоже с возможностью добавления новых монстров, очевидно), с которым герой будет сражаться. Во время сражения неплохо предоставлять игроку выбор вида "нанести удар - выпить зелье", а сам урон от удара можно немного рандомизировать с помощью одноименного модуля. Можно еще реализовать и метод sleep для мирного времени, но во время боя он, конечно, не понадобится. 

In [58]:
import random
# общий класс "герой"

class Hero:
    def __init__(self, name):
        self.name = name
        self.health = 1
        self.strength = 0.3
        self.mana = 0.3
    
    def __str__(self):
        pass
    
    # метод, кт. проверяет, осталось ли у игрока здоровье
    def __bool__(self):
        return True if (self.health > 0) else False
    
    # урон вычисляется по формуле здоровье ударенного = (сила удара бьющего/ мана ударенного),
    def hit(self, other):
        other.health -= (self.strength/(other.mana*10))
        other.mana -= 0.1
        other.health = round(other.health, 2)
        print(f"{self.name} hits {other.name}; ")
    
    def take_smth(self, smth, backpack):
        if smth in backpack and backpack.items[smth] > 0:
            setattr(self, smth, getattr(self, smth) + min(0.1, backpack.items[smth]))
            backpack.items[smth] = round(backpack.items[smth] - min(0.1, backpack.items[smth]), 2)
            print(f"your {smth} is {getattr(self, smth)} now, there is {backpack[smth]} gallons of {backpack.names(smth)} left;  ")
        else:
            print(f'you are out of {backpack.names(smth)}, don\'t loose your time!')
        
class Backpack():    
    def __init__(self):
        self.size = 0.5
        self.items = {}
        
    def __iter__(self):
        for item in self.items:
            yield (item, self.items[item])
            
    def names(self, key):
        names_dict = {'health': 'water', 
             'mana': 'berries'}
        return names_dict[key]
        
    def __contains__(self, x):
        return True if x in self.items else False
    
    def __getitem__(self, x):
        return self.items[x]
    
    def __str__(self):
        str_gen = f'size: {self.size}'
        for item in self.items:
            str_gen += f',  {self.names(item)}: {self[item]}'
        return str_gen
    
    def add(self, smth, amount):
        if amount > 0:
            if self.size - sum(self.items.values()) > 0 :
                if smth in self.items:
                    self.items[smth] += min(amount, self.size - sum(self.items.values()))
                else:
                    self.items[smth] = min(amount, self.size - sum(self.items.values()))
            else:
                return ('Attention, no more space in the backpack!')
        else:
            return ('Invalid number!')
    
# класс "боец"
class Fighter(Hero):
    def __init__(self, name):
        Hero.__init__(self, name)
        self.strength += 0.3
    
    def __str__(self):
        return f'{self.name}: health: {self.health}, strength: {self.strength}, mana: {self.mana}'
    
    def __repr__(self):
        return f"Fighter('{self.name}', {self.health}, {self.armor}, {self.strength})"
    
# класс "боец"
class Wizard(Hero):
    def __init__(self, name):
        Hero.__init__(self, name)
        self.mana += 0.3
    
    def __str__(self):
        return f'{self.name}: health: {self.health}, strength: {self.strength}, mana: {self.mana}'
    
    def __repr__(self):
        return f"Wizard('{self.name}', {self.health}, {self.armor}, {self.strength})"

        
class Monster():
    def __init__(self, name, strength, mana):
        self.name = name
        self.health = 1
        self.strength = strength
        self.mana = mana
        
    def __bool__(self):
        return True if (self.health > 0) else False
    
    def __str__(self):
        return f'{self.name}: health: {self.health}, strength: {self.strength}, mana: {self.mana}'
    
    def hit(self, other):
        other.health -= (self.strength/(other.mana*10))
        other.mana -= 0.1
        other.health = round(other.health, 2)
        print(f"{self.name} hits {other.name}; ")

# хотела выбирать направление с помощью стрелок, но так и не нашла, как это сделать, поэтому сделала на кнопках

def battle(hero, monster):
    hero.hit(monster) if hero.strength >= monster.strength else monster.hit(hero)
    print(hero)
    print(monster)

#     you\ monster: 0(attack from the front)    1(attack from the left)    2(attack from the left)
#     i - UP        battle                      have some time             have some time
#     j - LEFT      have some time              battle                     have some time
#     l - RIGHT     have some time              have some time             battle

def fight(hero, monster): # when the direction of your move coincides with the direction of monster's move, a battle begins
#     print('You are walking through the Haunted forest. There is a monster!')
    direction = input('your move: i-UP, j-LEFT, l-RIGHT:')
    monsters_move = random.randint(0,2)
    if (direction.lower() == 'i' and monsters_move == 0): # monsters_move = 0 - attack from the front
        battle(hero, monster)
    elif(direction.lower() == 'j' and monsters_move == 1): # monsters_move = 1 - attack from the left
        battle(hero, monster)
    elif(direction.lower() == 'l' and monsters_move == 1): # monsters_move = 1 - attack from the left
        battle(hero, monster)
    else:
        print(f'you have some time, your health is {hero.health}, your mana is {hero.mana}')
        action = input('what do you prefer: to attack first (a), or to look for some water(w)/berries(b)?')
        if action == 'a':
            hero.hit(monster)
            print(hero)
            print(monster)
        elif action == 'w':
            hero.take_smth("health", backpack)
        elif action == 'b':
            hero.take_smth("mana", backpack)
                  

# начать игру?
s = input("Would you like to play? Press enter if yes, press any key if no: ")
to_play = True if (s == '') else False

if to_play:
    s = input('Choose your hero: f - fighter, w - wizard: ')
    if (s == 'f'):
        name = input('input the name of your fighter: ')
        hero = Fighter(name)
    elif (s == 'w'):
        name = input('input the name of your wizard: ')
        hero = Wizard(name)
    print('You have a backpack of size 0.5 gallons')
    backpack = Backpack()
    water = input('How much water (increases heatlh) would you like to take with you? ')
    backpack.add("health", eval(water))
    berries = input('Hom much berries (increases mana) would you like to take with you? ')
    backpack.add("mana", eval(berries))
    print(backpack)
    
    print('now you are in the Haunted Forest, and there is a monster coming to you!')
    monster = Monster('big wolf', 1, 0.9)
    
    print('now your turn!')
    while to_play:
        if hero.health <= 0:
            print('GAME IS OVER, YOU ARE DEAD!!!!! BEEP BEEP BEEP')
            to_play = False
        elif monster.health <= 0:
            print('GAME IS OVER, YOU WON!!!!! CONGRATULATIONS')
            to_play = False
        else:
            print('')
            fight(hero, monster)

Would you like to play? Press enter if yes, press any key if no: 
Choose your hero: f - fighter, w - wizard: f
input the name of your fighter: petya
You have a backpack of size 0.5 gallons
How much water (increases heatlh) would you like to take with you? 0.3
Hom much berries (increases mana) would you like to take with you? 0.2
size: 0.5,  water: 0.3,  berries: 0.2
now you are in the Haunted Forest, and there is a monster coming to you!
now your turn!

your move: i-UP, j-LEFT, l-RIGHT:i
big wolf hits petya; 
petya: health: 0.67, strength: 0.6, mana: 0.19999999999999998
big wolf: health: 1, strength: 1, mana: 0.9

your move: i-UP, j-LEFT, l-RIGHT:i
you have some time, your health is 0.67, your mana is 0.19999999999999998
what do you prefer: to attack first (a), or to look for some water(w)/berries(b)?b
your mana is 0.3 now, there is 0.1 gallons of berries left;  

your move: i-UP, j-LEFT, l-RIGHT:i
you have some time, your health is 0.67, your mana is 0.3
what do you prefer: to attack 

In [40]:
d = {'foo': 1, 'boo' : 2, 'zoo' : 3}
for item in d:
    print(item)

foo
boo
zoo


In [81]:
class Backpack():
    def __init__(self):
        self.size = 0.5
        self.items = {}
        
    def __iter__(self):
        for item in self.items:
            yield (item, self.items[item])
        
    def __contains__(self, x):
        return True if x in self.items else False
    
    def __getitem__(self, x):
        return self.items[x]
    
#     def __setitem__(self, x):
#         self.items[x]
    
    def add(self, smth, amount):
        if amount > 0:
            if self.size - sum(self.items.values()) > amount :
                if smth in self.items:
                    self.items[smth] += amount
                else:
                    self.items[smth] = amount
            elif self.size - sum(self.items.values()) == 0:
                return ('Attention, no more space in the backpack!')
            else:
                print(f'there is only {self.size - sum(self.items.values())} space left, you added some {smth}!')
                if smth in self.items:
                    self.items[smth] += self.size - sum(self.items.values())
                else:
                    self.items[smth] = self.size - sum(self.items.values())
        else:
            return ('Invalid number!')
        
    def __str__(self):
        return f'size: {self.size}, {self.items}'

backpack = Backpack()
backpack.items
for i in backpack:
    print(i)
    
backpack.add("water", 0.3)
# backpack.add("berries", 0.3)

    
backpack.add('cookies', 0.3)
for i in backpack:
    print(i)
    
print(backpack)
backpack.items["water"] = round(backpack.items["water"] - 0.1,2)
print(backpack)

there is only 0.2 space left, you added some cookies!
('water', 0.3)
('cookies', 0.2)
size: 0.5, {'water': 0.3, 'cookies': 0.2}
size: 0.5, {'water': 0.2, 'cookies': 0.2}


#### Задача 2 (3 балла). 

Вспомним задачу токенизации. Напишите собственный простенький токенизатор (с самим процессом можно не сильно заморачиваться), который будет создавать генератор с объектами класса Token, у которых будет атрибут text и атрибут category (латинское слово, кириллическое слово или пунктуация). Токенизатор должен быть реализован в классе, у которого должна быть (генераторная) функция tokenize(). Вам понадобится отдельный класс для токенов и re.finditer(). 

In [25]:
import re

class Token:
    def __init__(self, text, category):
        self.text = text
        self.category = category 
        
    def __str__(self):
        return f'{self.text}: {self.category}'
    
    def __repr__(self):
        return f"Token('{self.text}', '{self.category}')"
    
class Tokenizer:
    def __init__(self, text):
        self.text = text
        
    def __iter__(self):
        return self.tokenize()
    
    def categories(self, grindex):
        cat = {1: 'cyrillic',
               2: 'latin',
               3: 'punctuation'}
        return cat[grindex]
        
    def tokenize(self):
        tokens = re.finditer(r'([а-яА-Я]+)|([a-zA-Z]+)|([^\w\s])', self.text)
        for t in tokens:
            for grindex in range(1,4):
                if t.group(grindex):
#                     print(f"token-la {t.group(grindex)} category: {self.categories(grindex)}")
#                     token = Token(t.group(grindex), self.categories(grindex))
                    yield Token(t.group(grindex), self.categories(grindex))
    
text = 'Жили-были dva гуся: вот и skazka вся.'
y = Tokenizer(text)
# y.tokenize()
for t in y:
    print(t)

Жили: cyrillic
-: punctuation
были: cyrillic
dva: latin
гуся: cyrillic
:: punctuation
вот: cyrillic
и: cyrillic
skazka: latin
вся: cyrillic
.: punctuation


#### Задача 3 (необязательная: + 3 балла + можно сдавать позднее дедлайна)

Самостоятельно изучите библиотеку pyqt5 и напишите программу - графический интерфейс для морфопарсинга: можно использовать любой морфопарсер. В программе должно быть окно, куда пользователь может вставлять свой собственный текст, кнопка "parse" и второе окно, в котором будет появляться морфоразбор. В каком виде он будет появляться, не принципиально, но можно сделать таблички. 

In [ ]:
# from PyQt5.QtWidgets import *
# import sys


# class MainWindow(QMainWindow): # главное окно
#     def __init__(self, parent=None):
#         super().__init__(parent)
#         self.setupUi()
#     def setupUi(self):
#         self.setWindowTitle("Hello, world") # заголовок окна
#         self.move(300, 300) # положение окна
#         self.resize(200, 200) # размер окна
#         self.lbl = QLabel('Hello, world!!!', self)
#         self.lbl.move(30, 30)


# if __name__ == "__main__":
#     app = QApplication(sys.argv)
#     win = MainWindow()
#     win.show()
#     sys.exit(app.exec_())

# from PyQt5.QtWidgets import QApplication, QLabel
# app = QApplication([])
# label = QLabel('Hello World!')
# label.show()
# app.exec_()

# from PyQt5.QtWidgets import *
# # from PyQt5.QtWidgets import QApplication, QWidget, QPushButton, QVBoxLayout
# app = QApplication([])
# window = QWidget()
# layout = QVBoxLayout()
# layout.addWidget(QTextEdit())
# layout.addWidget(QPushButton('Parse'))
# window.setLayout(layout)
# window.show()
# app.exec_()

from PyQt5 import QtWidgets
from PyQt5.QtWidgets import QApplication, QMainWindow
import sys

import pymorphy2


import re

class Token:
    def __init__(self, text, category):
        self.text = text
        self.category = category 
        
    def __str__(self):
        return f'{self.text}: {self.category}'
    
    def __repr__(self):
        return f"Token('{self.text}', '{self.category}')"
    
class Tokenizer:
    def __init__(self, text):
        self.text = text
        
    def __iter__(self):
        return self.tokenize()
    
    def categories(self, grindex):
        cat = {1: 'cyrillic',
               2: 'latin',
               3: 'punctuation'}
        return cat[grindex]
        
    def tokenize(self):
        tokens = re.finditer(r'([а-яА-Я]+)|([a-zA-Z]+)|([^\w\s])', self.text)
        for t in tokens:
            for grindex in range(1,4):
                if t.group(grindex):
#                     print(f"token-la {t.group(grindex)} category: {self.categories(grindex)}")
#                     token = Token(t.group(grindex), self.categories(grindex))
                    yield Token(t.group(grindex), self.categories(grindex))


class MyWindow(QMainWindow):
    def __init__(self):
        super(MyWindow,self).__init__()
        self.initUI()

    def button_clicked(self):
#         print("clicked")
#         self.label.setText("clicked!")
        yourtext = self.text1.toPlainText()
#         tokens = word_tokenize(yourtext)
        ytext = Tokenizer(yourtext)
        morph = pymorphy2.MorphAnalyzer()
        for t in ytext:
            if t.category == 'cyrillic':
                self.text2.append(f"{t.text}: {morph.parse(t.text)[0].tag.cyr_repr}")
#                 print(f"{t.text}: {morph.parse(t.text)[0].tag.cyr_repr}")
    
#         self.text2.setPlainText(text)

    def initUI(self):
        self.setGeometry(200, 200, 630, 300)
        self.setWindowTitle("Morphoparser")
        
        self.label1 = QtWidgets.QLabel(self)
        self.label1.setText("Insert your text here:")
        self.label1.move(10,0)
        self.label1.resize(200,30)
        
        self.text1 = QtWidgets.QTextEdit(self)
        self.text1.move(10,30)
        self.text1.resize(300,200)
        
        self.label2 = QtWidgets.QLabel(self)
        self.label2.setText("Parser:")
        self.label2.move(320,0)
        self.label2.resize(200,30)
        
        self.text2 = QtWidgets.QTextEdit(self)
        self.text2.move(320,30)
        self.text2.resize(300,200)
        
        self.b1 = QtWidgets.QPushButton(self)
        self.b1.setText("Parse")
        self.b1.move(10,240)
        self.b1.resize(200,30)
        self.b1.clicked.connect(self.button_clicked)


# # Reading the data from the first
# # textEdit and adding 10 to it
# result = self.textEdit1.toPlainText()+10


# # Displaying the result in the second textEdit.
# self.textEdit2.setPlainText(str(result)) 


def window():
    app = QApplication(sys.argv)
    win = MyWindow()
    win.show()
    sys.exit(app.exec_())

window()


In [22]:
# from nltk import sent_tokenize, word_tokenize

import pymorphy2
morph = pymorphy2.MorphAnalyzer()
# morph.parse('приемами')
text = 'приемами, с которыми они жили'

tokens = word_tokenize(text)
y = Tokenizer(text)
# y.tokenize()
for t in y:
    if t.category == 'cyrillic':
        print(f"{t.text}: {morph.parse(t.text)[0].tag.cyr_repr}")
        
# morph.parse('варусянка')[0].tag.cyr_repr
# morph.parse(tokens)
# morph.parse[0]
# print(morph.tag('приемами, с которыми'))

приемами: СУЩ,неод,мр мн,тв
с: ПР
которыми: ПРИЛ,субст?,мест-п,анаф мн,тв
они: МС,3л,анаф мн,им
жили: ГЛ,несов,неперех мн,прош,изъяв


    код для этой задачи удобнее будет писать в py. 